## Setup

In [1]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
from lightgbm import LGBMClassifier
from statistics import mode, stdev
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler, Normalizer

## Load dataset

In [2]:
info_cols_drop = ['Unnamed: 0']
work_cols_drop = ['Unnamed: 0', 'id', 'from_date', 'to_date', 'job/role', 'job_role_fillNan']

train_info = pd.read_csv("data/processed_data/train_info.csv").drop(info_cols_drop, axis=1)
train_work = pd.read_csv("data/processed_data/train_work.csv").drop(work_cols_drop, axis=1)
train_label = pd.read_csv("toan-dataset/label_train.csv")

test_info = pd.read_csv("data/processed_data/test_info.csv").drop(info_cols_drop, axis=1)
test_work = pd.read_csv("data/processed_data/test_work.csv").drop(work_cols_drop, axis=1)
test_label = pd.read_csv("toan-dataset/label_test.csv")

train_work

,id_bh,new_work_province_id,employee_lv,year_from_date,year_to_date,month_from_date,month_to_date,num_year_contract,num_month_contract,company_type,id_management,id_office,job_role_encode_knn
0,113039360,24,7.0,2013,2015,1,12,2,35,-1,6.0,7603.0,12
1,113039360,24,10.0,2016,2022,1,4,6,75,-1,6.0,7603.0,12
2,116074930,24,7.0,2016,2016,6,12,0,6,-1,2.0,6077.0,7
3,116074930,24,8.0,2017,2017,1,3,0,2,-1,2.0,6077.0,7
4,116074930,-999,-1.0,2017,2017,4,7,0,3,-1,2.0,-999.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
247554,2616117553,-999,2.0,2010,2010,8,9,0,1,-1,97.0,11274.0,4
247555,2616117553,-999,-1.0,2010,2016,10,10,6,72,-1,97.0,-999.0,1
247556,2616117553,62,6.0,2016,2017,11,12,1,13,1,97.0,10840.0,108
247557,2616117553,62,8.0,2018,2019,1,12,1,23,1,97.0,10840.0,108


## Function

In [3]:
def max_frequent(list):
    if (len(list) == 0):
        return -999
    return(mode(list))
    
def max_frequent_without_filled_var(list):
    if( -999 in list):
        list.remove(-999)
    if(-1 in list):
        list.remove(-1)
    if(-1.0 in list):
        list.remove(-1.0)
    a_set = set(list)
    if ((len(list) == 0) or (len(a_set) == len(list)) ):
        return -999
    return(mode(list))

def last_value(list):
    if (len(list) == 0):
        return -999
    return list[-1]

def count_unique(list):
    return len(set(list))

def count_unique_without_filled_var(list):
    if( -999 in list):
        list.remove(-999)
    if(-1 in list):
        list.remove(-1)
    return len(set(list))

def min_without_filled_var(list):
    if(-1 in list):
        list.remove(-1)
        if (len(list) == 0):
            return -1
        else: 
            return min(list)
    return min(list)
    
def std(list):
    if (len(list) < 2):
            return -999
    if(-1 in list):
        list.remove(-1)
        if (len(list) < 2):
            return -999
        else: 
            return stdev(list)
    return stdev(list)

In [4]:
def feature_scaling(df_train, df_test, cols, index_scaler=0):
    list_scaler = [MinMaxScaler(), StandardScaler(), Normalizer()]
    scaler = list_scaler[index_scaler]
    df_train[cols] = scaler.fit_transform(df_train[cols])
    df_test[cols] = scaler.transform(df_test[cols])  
    return df_train, df_test

## Group and merge data

In [5]:
def merge_data(df, col='id_bh'):
    df = df.groupby(col).agg(lambda x: x.tolist())
    return df
    
def concat_data(df, df_add, col='id_bh', how='left'):
    df = pd.merge(df, df_add, on=col, how=how)
    return df

def creat_train_and_test(train_info, train_work, train_label, test_info, test_work):
    train = merge_data(train_work, col='id_bh')
    train = concat_data(train, train_info, col='id_bh', how='left')
    train = concat_data(train, train_label, col='id_bh', how='left')

    test = merge_data(test_work, col='id_bh')
    test = concat_data(test, test_info, col='id_bh', how='left')

    return train, test


In [6]:
train, test = creat_train_and_test(train_info.copy(), train_work.copy(), train_label.copy(), test_info.copy(), test_work.copy())
print(train.shape, test.shape)
train

(27502, 17) (18134, 16)


,id_bh,new_work_province_id,employee_lv,year_from_date,year_to_date,month_from_date,month_to_date,num_year_contract,num_month_contract,company_type,id_management,id_office,job_role_encode_knn,bithYear,gender,new_province_id,label
0,100000725,"[-999, -999, -999, -999, -999, -999, -999, -99...","[-1.0, -1.0, -1.0, 0.0, 4.0, 5.0, 6.0, 6.0, 7....","[1983, 1987, 1990, 1992, 1993, 1996, 2000, 200...","[1987, 1990, 1992, 1993, 1996, 2000, 2004, 200...","[3, 3, 6, 9, 4, 9, 5, 4, 9, 10, 5, 1, 9, 1, 9,...","[2, 5, 8, 3, 8, 4, 3, 8, 9, 4, 12, 8, 12, 8, 2...","[4, 3, 2, 1, 3, 4, 4, 0, 0, 3, 1, 1, 1, 1, 3, ...","[47, 38, 26, 6, 40, 43, 46, 4, 0, 30, 19, 19, ...","[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[2381.0, 2381.0, 2381.0, 2381.0, 2381.0, 2381....","[55, 265, 104, 68, 68, 68, 22, 23, 23, 159, 1,...",1963,1,24,5
1,100007067,"[-999, -999, 3, -999, 3, 3, 3, 3, 3, 3]","[4.0, 5.0, 5.0, -1.0, 5.0, 7.0, 8.0, 7.0, 8.0,...","[1997, 2004, 2008, 2010, 2010, 2012, 2015, 201...","[2004, 2008, 2010, 2010, 2012, 2015, 2015, 201...","[10, 10, 12, 11, 12, 2, 2, 1, 1, 1]","[9, 11, 10, 11, 1, 1, 12, 12, 12, 4]","[7, 4, 2, 0, 2, 3, 0, 0, 1, 3]","[83, 49, 22, 0, 13, 35, 10, 11, 23, 39]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[76.0, 76.0, 76.0, 76.0, 76.0, 76.0, 76.0, 76....","[4288.0, 4288.0, 4288.0, 4288.0, 4288.0, 4288....","[4, 4, 179, 1, 179, 179, 179, 179, 179, 179]",1971,1,3,4
2,100007555,"[-999, -999, -999, -999, -999, -999, -999, -99...","[-1.0, -1.0, 5.0, 5.0, 6.0, 5.0, 6.0, 9.0, 11....","[1989, 1991, 1994, 1997, 2000, 2003, 2004, 200...","[1991, 1993, 1997, 2000, 2003, 2003, 2004, 200...","[3, 7, 1, 3, 12, 7, 1, 10, 12, 3, 6, 1, 1, 6, ...","[6, 12, 2, 11, 6, 12, 9, 11, 2, 5, 12, 12, 5, ...","[2, 2, 3, 3, 3, 0, 0, 2, 4, 0, 0, 2, 1, 0, 0, ...","[27, 29, 37, 44, 30, 5, 8, 25, 38, 2, 6, 35, 1...","[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -...","[9.0, 9.0, 9.0, 9.0, 9.0, 9.0, 9.0, 9.0, 9.0, ...","[4223.0, 4223.0, 4223.0, 4223.0, 4223.0, 4223....","[55, 7, 91, 102, 102, 102, 102, 102, 102, 42, ...",1970,1,-999,2
3,100008102,"[24, 24, 24, 24, 24, 24, 24, 24, 24, 63, 63, 6...","[5.0, 6.0, 6.0, 7.0, 8.0, 9.0, 9.0, 10.0, 13.0...","[1996, 2001, 2004, 2004, 2004, 2006, 2007, 200...","[2001, 2004, 2004, 2004, 2006, 2007, 2008, 201...","[12, 12, 5, 10, 12, 12, 10, 12, 2, 4, 2, 1, 5,...","[11, 4, 9, 11, 11, 9, 11, 1, 3, 1, 12, 4, 11, ...","[5, 3, 0, 0, 2, 1, 1, 2, 1, 1, 0, 0, 0, 0, 1, ...","[59, 28, 4, 1, 23, 9, 13, 13, 13, 9, 10, 3, 6,...","[-1, -1, -1, -1, -1, -1, -1, 6, 6, -1, -1, -1,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[2335.0, 2335.0, 2335.0, 2335.0, 2335.0, 2335....","[112, 112, 40, 40, 40, 40, 64, 64, 291, 291, 2...",1970,1,23,5
4,100008777,"[-999, -999, -999, -999, -999, -999, -999, -99...","[0.0, 5.0, 6.0, 6.0, 7.0, 7.0, 9.0, 10.0, 10.0...","[1990, 1993, 1996, 1999, 2002, 2004, 2004, 200...","[1993, 1996, 1998, 2001, 2004, 2004, 2004, 200...","[6, 4, 3, 1, 1, 2, 10, 1, 10, 1, 7, 3, 1, 4, 1...","[3, 2, 12, 12, 1, 9, 12, 9, 12, 6, 2, 12, 3, 1...","[3, 3, 2, 2, 2, 0, 0, 0, 0, 0, 1, 0, 2, 0, 2, ...","[33, 34, 33, 35, 24, 7, 2, 8, 2, 5, 7, 9, 26, ...","[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -...","[14.0, 14.0, 14.0, 14.0, 14.0, 14.0, 14.0, 14....","[10755.0, 10755.0, 10755.0, 10755.0, 10755.0, ...","[137, 137, 137, 48, 48, 12, 12, 12, 12, 48, 21...",1965,1,-999,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27497,9719632107,"[-999, -999, -999, 24, -999, -999]","[12.0, 13.0, -1.0, 9.0, 12.0, 14.0]","[2019, 2020, 2020, 2020, 2021, 2021]","[2019, 2020, 2020, 2020, 2021, 2021]","[7, 1, 3, 6, 1, 7]","[12, 2, 5, 12, 6, 12]","[0, 0, 0, 0, 0, 0]","[5, 1, 2, 6, 5, 5]","[-1, -1, -1, -1, -1, -1]","[28.0, 28.0, 28.0, 28.0, 28.0, 28.0]","[3560.0, 3560.0, -999.0, 4054.0, 4054.0, 4054.0]","[1, 1, 1, 29, 29, 29]",1995,1,-999,6
27498,9809227388,"[-999, -999, -999, 40, 40, 40, 40, 40, -999, 40]","[6.0, 9.0, 10.0, 10.0, 11.0, 9.0, 7.0, 7.0, 7....","[2009, 2010,

## Feature engineering

In [7]:
category_cols = [
    "company_type",	
    "id_management", 
    "id_office", 
    "job_role_encode_knn",
    "new_work_province_id",
]
numeric_cols = [
    "employee_lv" , 
    "year_from_date",
    "year_to_date",
    "month_from_date",
    "num_year_contract",
    "num_month_contract",
]

def features_engineering(df):
    for col in category_cols:
        df['category_max_fre_' + col] = df[col].apply(lambda x: max_frequent(x) ) ## category
        df['category_max_fre_out_filled_var_' + col] = df[col].apply(lambda x: max_frequent_without_filled_var(x) ) ## category
        df['category_last_value_' + col] = df[col].apply(lambda x: last_value(x) ) ## category
        df['numeric_count_unique_' + col] = df[col].apply(lambda x: count_unique(x) ) ## numeric
        df['numeric_count_unique_without_filled_var' + col] = df[col].apply(lambda x: count_unique_without_filled_var(x) ) ## numeric

    for col in numeric_cols:
        if (col == 'employee_lv'):
            df['numeric_max_lv_' + 'of_'+ col] = df[col].apply(lambda x: max(x) )
            df['numeric_min_lv_' + 'of_'+ col] = df[col].apply(lambda x: min_without_filled_var(x) )
            df['numeric_mode_' + 'of_'+ col] = df[col].apply(lambda x: max_frequent(x) ) 
            df['numeric_mode_out_filled_var_' + 'of_'+ col] = df[col].apply(lambda x: max_frequent_without_filled_var(x) ) 
            df['numeric_mean_' + 'of_'+ col] = df[col].apply(lambda x: sum(x) / float(len(x)) ) 
            df['numeric_std_' + 'of_'+ col] = df[col].apply(lambda x: std(x) )
            df['numeric_count_unique_' + 'of_'+ col] = df[col].apply(lambda x: count_unique(x) ) ## numeric
            df['numeric_count_unique_' + 'of_'+ col] = df[col].apply(lambda x: count_unique_without_filled_var(x) ) ## numeric
            df['numeric_Q25_' + 'of_'+ col ] = df[col].apply(lambda x: np.quantile(x, .25) )
            df['numeric_Q75_' + 'of_'+ col ] = df[col].apply(lambda x: np.quantile(x, .75) )
        else:
            df['numeric_max_' + 'of_'+ col] = df[col].apply(lambda x: max(x) )
            df['numeric_min_' + 'of_'+ col] = df[col].apply(lambda x: min_without_filled_var(x) )
            df['numeric_mode_' + 'of_'+ col] = df[col].apply(lambda x: max_frequent(x) )  
            df['numeric_mean_' + 'of_'+ col] = df[col].apply(lambda x: sum(x) / float(len(x)) ) 
            df['numeric_std_' + 'of_'+ col] = df[col].apply(lambda x: std(x) )
            df['numeric_count_unique_' + 'of_'+ col] = df[col].apply(lambda x: count_unique(x) ) ## numeric
            df['numeric_Q25_' + 'of_'+ col ] = df[col].apply(lambda x: np.quantile(x, .25) )
            df['numeric_Q75_' + 'of_'+ col ] = df[col].apply(lambda x: np.quantile(x, .75) )

    
    df['numeric_year_diff_max_year_from_vs_bithYear'] = df['numeric_max_of_year_from_date'] - df['bithYear']
    df['numeric_year_diff_min_year_from_vs_bithYear'] = df['numeric_min_of_year_from_date'] - df['bithYear']
    df['numeric_year_diff_max_year_to_vs_bithYear'] = df['numeric_max_of_year_to_date'] - df['bithYear']
    df['numeric_year_diff_min_year_to_vs_bithYear'] = df['numeric_min_of_year_to_date'] - df['bithYear']

    df['numeric_olds'] = 2022 - df['bithYear']

    df['numeric_ratio_olds_vs_(diff_min_year_from_vs_bithYear)'] = df['numeric_olds'] / df['numeric_year_diff_min_year_from_vs_bithYear']
    df['numeric_ratio_olds_vs_(diff_min_year_from_vs_bithYear)'] = df['numeric_ratio_olds_vs_(diff_min_year_from_vs_bithYear)'].apply(lambda x: x if(x> 0) else -999)

    ## fix value in create_new_fea cols
    new_cols = [col for col in df.columns if 'numeric_year_diff_' in col]
    for col in new_cols:
        df[col] = df[col].apply(lambda x: x if(x> 0) else -999)

    return df


In [8]:
fe_train = features_engineering(train.copy())
fe_test = features_engineering(test.copy())

## Pretrain

In [9]:
numeric_ml_useful_cols = [col for col in fe_train.columns if 'numeric_' in col]
category_ml_useful_cols = [col for col in fe_train.columns if 'category_' in col]
old_cols = ["bithYear",	"gender", "new_province_id", "id_bh", "label"]

ml_useful_cols_train = numeric_ml_useful_cols + category_ml_useful_cols + old_cols
ml_useful_cols_test = numeric_ml_useful_cols + category_ml_useful_cols + old_cols[:-1]

In [10]:
category_cols = category_ml_useful_cols + ["new_province_id"]
numeric_cols = numeric_ml_useful_cols[:-1] + ["bithYear"]

df_train = fe_train[ml_useful_cols_train]
df_test = fe_test[ml_useful_cols_test]

In [11]:
X_train = df_train.drop(columns = ['label', 'id_bh']) 
X_train[category_cols] = X_train[category_cols].astype('category')
y_train = df_train['label']

X_test = df_test.drop(columns = ['id_bh']) 
X_test[category_cols] = X_test[category_cols].astype('category')
X_train.shape, X_test.shape


((27502, 83), (18134, 83))

## Parameters

In [12]:
params_k = {
            'boosting_type': 'gbdt',
            'objective': 'multiclass',
            'num_class':7,
            'categorical_feature': category_cols,
            'verbose' : 0,
            'force_col_wise': True,
            "random_seed":42,
            # 'class_weight': class_weights_dict,
            # 'subsample': 0.5,
            # 'subsample_freq': 1,
            # 'learning_rate': 0.01,
            # 'num_leaves': 2**11-1,
            # 'min_data_in_leaf': 2**12-1,
            # 'feature_fraction': 0.5,
            # 'max_bin': 100,
            # 'n_estimators': 500,
            # 'boost_from_average': False,
            # "feature_name": X_train.columns.tolist(),
}

## Cross-validation

In [13]:
skf = StratifiedKFold(n_splits= 5, random_state= 42, shuffle= True)
cv_score = []
## Kfold
for n_fold, (train_idx, valid_idx) in enumerate(skf.split(X_train, y_train)):
    print("Fold %s" % (n_fold))
    train_x, train_y = X_train.iloc[train_idx], y_train.iloc[train_idx]
    valid_x, valid_y = X_train.iloc[valid_idx], y_train.iloc[valid_idx]
    
    train_x, valid_x = feature_scaling(train_x, valid_x, numeric_cols)

    clf = LGBMClassifier(**params_k)
    clf.fit(train_x, train_y, verbose= False)

    score = f1_score(valid_y, clf.predict(valid_x),  average='macro')
    cv_score.append(score)
    print('F1_SCORE: ', round(score, 4), '\n')

print(sum(cv_score)/ 5)
print(std(cv_score))


Fold 0
[LightGBM] [Warning] categorical_feature is set=category_max_fre_company_type,category_max_fre_out_filled_var_company_type,category_last_value_company_type,category_max_fre_id_management,category_max_fre_out_filled_var_id_management,category_last_value_id_management,category_max_fre_id_office,category_max_fre_out_filled_var_id_office,category_last_value_id_office,category_max_fre_job_role_encode_knn,category_max_fre_out_filled_var_job_role_encode_knn,category_last_value_job_role_encode_knn,category_max_fre_new_work_province_id
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
F1_SCORE:  0.7498 

Fold 1
[LightGBM] [Warning] categorical_feature is set=category_max_fre_company_type,category_max_fre_out_filled_var_company_type,category_last_value_company_type,category_max_fre_id_management,category_max_fre_out_filled_var_id_management,category_last_value_id_management,category_max_fre_id_office,category_max_fre_out_filled_var_id_office,category_last_value_id

## Predict test

In [14]:
X_train_scale, X_test_scale = feature_scaling(X_train.copy(), X_test.copy(), numeric_cols)
clf = LGBMClassifier(**params_k)
clf.fit(X_train_scale, y_train)
y_pred = clf.predict(X_test_scale)

[LightGBM] [Warning] categorical_feature is set=category_max_fre_company_type,category_max_fre_out_filled_var_company_type,category_last_value_company_type,category_max_fre_id_management,category_max_fre_out_filled_var_id_management,category_last_value_id_management,category_max_fre_id_office,category_max_fre_out_filled_var_id_office,category_last_value_id_office,category_max_fre_job_role_encode_knn,category_max_fre_out_filled_var_job_role_encode_knn,category_last_value_job_role_encode_knn,category_max_fre_new_work_province_id
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


In [15]:
df_pred = pd.DataFrame()
df_pred['id_bh'] = df_test['id_bh']
df_pred['label'] = y_pred

## merge by label_test
submit_file = concat_data(test_label, df_pred)
print(submit_file['label'].value_counts())
submit_file

2    6557
4    5423
3    2294
6    2052
5    1749
1      34
7      25
Name: label, dtype: int64


,id_bh,label
0,113118886,2
1,2521527855,2
2,2421701111,2
3,2620332660,2
4,2905004143,6
...,...,...
18129,2717061671,5
18130,2612005193,6
18131,7916334833,3
18132,2616146670,4


In [16]:
submit_file.to_csv('lgb_baseline_submission.csv', index=False)
print(f"Export sucessfully!")

Export sucessfully!
